In [2]:
import xarray as xr
import os

from cycler import cycler
from matplotlib import cm as cmap

In [16]:
# Get latest version dir.
dirs = [os.path.join('../lightning_logs/', d) for d in os.listdir('../lightning_logs/') if os.path.isdir(os.path.join('../lightning_logs/', d))]
log_dir = sorted(dirs, key=lambda x: os.path.getctime(x), reverse=True)[0]
log_dir

'../lightning_logs/version_5'

In [17]:
ds = xr.open_dataset(os.path.join(log_dir, 'predictions.nc'))

In [18]:
ds

<xarray.Dataset>
Dimensions:    (epoch: 10, iter: 100000, time: 17520)
Coordinates:
  * time       (time) datetime64[ns] 2007-01-01T00:15:00 ... 2007-12-31T23:45:00
  * epoch      (epoch) int64 0 1 2 3 4 5 6 7 8 9
  * iter       (iter) int64 0 1 2 3 4 5 ... 99994 99995 99996 99997 99998 99999
Data variables:
    reco       (time) float64 ...
    rb         (time) float64 ...
    reco_pred  (epoch, time) float64 ...
    rb_pred    (epoch, time) float64 ...
    q10        (iter) float32 ...